# HBA PA5

French version: https://github.com/Daniel-R-Armstrong/French-Wiki-2500-Pretrained-SentencePiece-LM/blob/master/fr_spm.ipynb
https://forums.fast.ai/t/sentencepiece/53010/50

# Turkish ULMFiT from scratch

In [ ]:
# #hide
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [88]:
import warnings; warnings.simplefilter('ignore')

## HBA: TextList is in fastaiv1
https://fastai1.fast.ai/text.data.html#TextList.from_folder
https://docs.fast.ai/text.data#TextBlock.from_folder

In [1]:
import sys
!{sys.executable} -m pip install fastai==1.0.61 #This right here

In [ ]:
# import sys
# !{sys.executable} -m pip install fastcore==1. #This right here

In [ ]:
# #fastai v1 API: https://fastai1.fast.ai/overview.html
# from fastai.basics import *

In [ ]:
# from fastai.utils import *
# show_install()

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [3]:
from fastai.imports import *

In [4]:
data_path = Path('/home/ec2-user/.fastai/data')

In [5]:
bs=128
# torch.cuda.set_device(0)
# data_path = Config.data_path()
# data_path = '/home/ec2-user/.fastai/data'

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [6]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

In [7]:
path

PosixPath('/home/ec2-user/.fastai/data/trwiki')

In [8]:
lang

'tr'

## Turkish wikipedia model

In [8]:
# #HBA
# import sys
# !{sys.executable} -m pip install wikiextractor

In [9]:
# !pip install wikiextractor

In [9]:
from nlputils import split_wiki,get_wiki

## Uncompress wiki file from cli

Source: https://github.com/attardi/wikiextractor

1. Do `python setup.py install`
1. Double check with: `which wikiextractor`

`
base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`

1. Uncrompress

`
(base) [ec2-user@ip-172-16-69-8 trwiki]$ python -m wikiextractor.WikiExtractor trwiki-latest-pages-articles.xml
INFO: 2832129   Mucize Dünyası: New York – Birleşik Kahramanlar
INFO: 2832111   Kraliyet Mühendislik Akademisi üyesi kadınlar listesi
INFO: 2832131   Ilana Rovina
INFO: 2832155   Artur Taymazov
INFO: 2832130   Enka (kadın voleybol takımı) 1992-93 sezonu
INFO: Finished 31-process extraction of 377951 articles in 123.1s (3071.5 art/s)
INFO: total of page: 732729, total of articl page: 377951; total of used articl page: 377951
(base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`
    


In [10]:
get_wiki(path,lang)

/home/ec2-user/.fastai/data/trwiki/trwiki already exists; not downloading


In [11]:
# from nlputils import split_wiki,get_wiki

# get_wiki(path,lang)
!head -n4 {path}/{name}

<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">
Cengiz Han

Cengiz Han (doğum Temuçin, – 18 Ağustos 1227) ölümünden sonra tarihin en büyük yüzölçümüne sahip imparatorluğu haline gelmiş Moğol İmparatorluğu'nun kurucusu ve ilk Kağanı ve İmparatoru olan Moğol komutan ve hükümdar. 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus hâline getirdi ve o ulusu "Moğol" siyasi kimliği çatısı altında topladı.


In [12]:
path.ls()

[PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/wikiextractor-orig'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki-latest-pages-articles.xml'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/log'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/wikiextractor'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/models')]

In [13]:
dest = split_wiki(path,lang)

/home/ec2-user/.fastai/data/trwiki/docs already exists; not splitting


In [14]:
dest.ls()[:5]

[PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Kara delik.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Heaviside katmanı.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Zurna.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Özallı Yıllar.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Kuşça, Cihanbeyli.txt')]

In [15]:
dest

PosixPath('/home/ec2-user/.fastai/data/trwiki/docs')

Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

# HBA fix

1. Open via: `vi ~/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py`
1. see: https://forums.fast.ai/t/multifit-runtime-error-permission-denied/72874/5
1. See: https://github.com/fastai/fastai/issues/2572 

In [16]:
#PRINT NUMBER OF FILES IN DIR
!find /home/ec2-user/.fastai/data/trwiki/docs -mindepth 1 -maxdepth 1 -printf '.' | wc -c

56219


# hba why it takes soo long, 2 hrs to create data
https://forums.fast.ai/t/getting-memoryerror-when-trying-to-create-textlist/40261/28

Quotes:

`
Yes there is a high peak of memory usage during the preprocessing, because python sucks at multi-process, so the array with the results is copied across all the processes instead of being shared.
To reduce RAM usage, reduce the number of workers (but then it’s going to be slower).
`

more 

`
Tokenizing the full dataset took about 2 hours.

When you were working through yours, did you max out your memory? I’ve found that if you try to tokenize too much at once and you don’t have enough memory to finish the process, it tends to stall and never complete. You might need to try working with a small chunk at a time.
`

more

`
It doesn’t always scale linearly when you train on multiple GPUs, so I can’t really say. Training WT103 with the standard AWD_LSTM is roughly 1h-1h30 an epoch IIRC, on a p3 (so a V100).

...

Ok so I finally got past the tokenization phase and have a databunch in hand!

It still took quite a bit of memory (peaked at 345GB of the 624 GB available on n1-highmem-96) but it was at least 2x less memory than the standard way with the default processors (which exhausted the available memory on this instance and crashed) and didn’t seem to take any longer (it didn’t seem to get as “stuck” halfway through either)
`

## Read About TextDataBunch
https://fastai1.fast.ai/text.data.html#TextDataBunch

### TextData

class TextList[source][test]
TextList(items:Iterator[T_co], vocab:Vocab=None, pad_idx:int=1, sep=' ', **kwargs) :: ItemList

Basic ItemList for text data.

vocab contains the correspondence between ids and tokens, pad_idx is the id used for padding. You can pass a custom processor in the kwargs to change the defaults for tokenization or numericalization. It should have the following form:

tokenizer = Tokenizer(SpacyTokenizer, 'en')
processor = [TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(max_vocab=30000)]
To use sentencepiece instead of spaCy (requires to install sentencepiece separately) you would pass

processor = SPProcessor()

### Preprocessor

PreProcessor that tokenizes and numericalizes with sentencepiece

pre_rules and post_rules default to defaults.text_pre_rules and defaults.text_post_rules respectively, vocab_sz defaults to the minimum between max_vocab_sz and one quarter of the number of words in the training texts (rounded to the nearest multiple of 8). model_type is passed to sentencepiece, so can be unigram (default), bpe, char, or word. Other sentencepiece parameters are langm max_sentence_len and char_coverage (default to 1. for European languages and 0.99 for others).

## CPU notes: 

Some tasks take up to 86GB x 4 = 320 GB. See Trello SS

- label_for_lm()
- databunch()
- fine_tune

## Quotes from 2020 NLP NB

"Subword tokenization provides a way to  scale between character tokenization  & word tokenization, & handles every human language without needing language-specific algorithms to be developed. It can even handle genomic sequences or MIDI music!" #NLP 

Our special rules tokens appear first, and then every word appears once, in frequency order. The defaults to Numericalize are min_freq=3,max_vocab=60000. max_vocab=60000 results in fastai replacing all words other than the most common 60,000 with a special unknown word token, xxunk. This is useful to avoid having an overly large embedding matrix, since that can slow down training and use up too much memory, and can also mean that there isn't enough data to train useful representations for rare words. However, this last issue is better handled by setting min_freq; the default min_freq=3 means that any word appearing less than three times is replaced with xxunk.

fastai can also numericalize your dataset using a vocab that you provide, by passing a list of words as the vocab parameter.

In [17]:
# data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
#         .split_by_rand_pct(0.1, seed=42)
#         .label_for_lm()
#         .databunch(bs=bs, num_workers=1))

# data.save(f'{lang}_databunch')
# len(data.vocab.itos),len(data.train_ds)

In [18]:
tl = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
        .split_by_rand_pct(0.1, seed=42))

In [19]:
tll = tl.label_for_lm()

/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py:475: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ds.items = np.array(sum(e.map(self._encode_batch, partition_by_cores(ds.items, self.n_cpus)), []))


In [20]:
data = tll.databunch(bs=bs, num_workers=1)

In [21]:
data.save(f'{lang}_databunch')

In [22]:
len(data.vocab.itos),len(data.train_ds)

(30000, 50586)

In [23]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [24]:
data.show_batch()

/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,"▁nitelen irler . ▁xxmaj ▁kara ▁delikler in , ▁"" teki l lik "" leri ▁dolayısıyla , ▁üç ▁boyutlu ▁olmadıkları , ▁sıfır ▁hacimli ▁oldukları ▁kabul ▁edilir . ▁xxmaj ▁kara de lik lerin ▁içinde ▁zamanın ▁ise ▁yavaş ▁ak tığı ▁veya ▁ak madığı ▁tahmin ▁edilmektedir . ▁xxmaj ▁kara ▁delikler ▁xxmaj ▁einstein ' ın ▁genel ▁görelilik ▁kuramı yla ▁tanımlanmış lardır . ▁xxmaj ▁doğrudan ▁gözlem len eme mekle ▁birlikte , ▁çeşitli ▁dalga ▁boy larını ▁kullanan"
1,"▁xxmaj ▁samsun ▁merkezli ▁xxmaj ▁ a de ka ▁da ▁hâlen ▁faaliyet te dir . ▁xxmaj ▁son ▁yıllarda ▁yapılan ▁yatırım larla ▁birlikte ▁enerji ▁kaynaklarının ▁xxmaj ▁türkiye ' ye ▁giriş ▁kapı larından ▁biri ▁durumuna ▁gelen ▁xxmaj ▁samsun ▁zengin ▁enerji ▁kaynaklarına ▁yakınlığı ▁ve ▁uluslararası ▁ticari ▁ilişkileri yle ▁çok ▁yönlü ▁bir ▁merkez ▁görünüm dedir . ▁xxmaj ▁samsun , ▁2003' te ▁açılan ▁xxmaj ▁mavi ▁xxmaj ▁akım ▁ile ▁birlikte ▁xxmaj ▁rusya ▁doğalgaz ının ▁xxmaj ▁türkiye '"
2,"▁xxmaj ▁vektör ▁formunda , ▁eğer ▁i , ▁gelen ▁ışın ▁yönünde ▁işaret ▁eden ▁bir ▁birim ▁vektör ▁ise ▁ve ▁ n , ▁yüzey in ▁normal i ▁ise , ▁kır ılan ▁ r ▁ ışını nın ▁yönü ▁şöyledir : ▁[ 3 ] ▁burada ▁ δ ▁aşağıdaki ▁ifade ye ▁eşittir . ▁xxmaj ▁eğer ▁i ▁ · ▁ n ▁< 0 ▁ise ▁hesaplamalar da ▁ - n ▁kullanılmalıdır . ▁ δ ▁< 0 ▁olduğunda ,"
3,"şarkıcı ) ▁xxmaj ▁kim ▁xxmaj ▁ki - bu m ▁( doğum ▁23 ▁xxmaj ▁eylül ▁1991 ) ▁ya ▁da ▁daha ▁çok ▁bilinen ▁sahne ▁adıyla ▁xxmaj ▁ key , ▁xxmaj ▁güney ▁xxmaj ▁koreli ▁şarkıcı ▁ve ▁oyuncu dur . ▁shin e e ▁grubunun ▁üyesidir . ▁xxmaj ▁ key , ▁23 ▁xxmaj ▁eylül ▁1991' de ▁xxmaj ▁da e gu , ▁xxmaj ▁güney ▁xxmaj ▁kore ' de ▁doğdu . ▁2006 ▁yılında ▁xxup ▁sm ▁xxmaj ▁entertainment"
4,"▁xxmaj ▁ayetullah ▁xxmaj ▁ebu ▁el - kasım ▁el - ho y ' un ▁yanında ▁okumak ▁üzere ▁xxmaj ▁irak ' ın ▁xxmaj ▁necef ▁kentine ▁gitmiştir . ▁xxmaj ▁ si stan i , ▁xxmaj ▁irak ▁xxmaj ▁cumhurbaşkanı ▁xxmaj ▁abdul ker im ▁xxmaj ▁kasım ▁yönetimindeki ▁askeri ▁jun ta ▁rejimi ▁döneminde ▁1960 ▁yılında ▁xxmaj ▁tak li d ▁xxmaj ▁ merci i ▁unvanına ▁ulaşmıştır . ▁xxmaj ▁ si stan i ▁henüz ▁31 ▁yaşındayken ▁(19 61"


In [25]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

### File Locations language model, vocab, databunch

In [ ]:
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/tmp/spm.model
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/tmp/spm.vocab
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/docs/tr_databunch

## About Jupyter going idle 
The default autosave interval in Jupyter notebook is 120 seconds. You can change this with the %autosave magic command for an individual notebook like this:

To change it for all of your notebooks, you need to modify the custom.js file.


1. Navigate to your jupyter config directory, which you can find by running the following command in the Terminal:

jupyter --config-dir

In [26]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

## For output of this Training job. See Tweet: 

- https://twitter.com/HBAkirmak/status/1322256689022775297
- https://twitter.com/HBAkirmak/status/1322148506069512193?s=20

In [29]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

In [30]:
learn.to_fp32().export('/home/ec2-user/.fastai/data/trwiki/models/learner_modtr_spm.pkl')

In [31]:
learn.data.vocab.save('/home/ec2-user/.fastai/data/trwiki/models/learner_vocab_tr_spm.pkl')


In [32]:
learn.save_encoder('/home/ec2-user/.fastai/data/trwiki/models/learner_tr_spm_enc')


In [33]:
learn.to_fp32().save('/home/ec2-user/.fastai/data/trwiki/models/learner_mod_tr_spm_save.pkl',return_path=True)


PosixPath('/home/ec2-user/.fastai/data/trwiki/models/learner_mod_tr_spm_save.pkl.pth')

# From NN-IMDB NB

In [34]:
len(data.vocab.itos),len(data.train_ds)

(30000, 50586)

In [35]:
data.show_batch()

idx,text
0,ing ▁xxmaj ▁da rt ▁xxmaj ▁operasyonu ' nu ▁başlatma ya ▁karar ▁verdi . ▁xxup ▁abd ▁bombalama sı ▁sırasında ▁hala ▁xxmaj ▁han o i ' de ▁bulunan ▁xxmaj ▁ko si gin ▁ve ▁xxmaj ▁sovyet ▁hükûmeti ▁xxmaj ▁kuzey ▁xxmaj ▁vietnam ' a ▁askeri ▁yardım larını ▁artırma ya ▁karar ▁verdi ▁ve ▁böylece ▁xxmaj ▁kruşçev ' in ▁xxmaj ▁vietnam ' daki ▁politikasının ▁büyük ▁oranda ▁tersine ▁döndüğü ▁dönem ▁başladı . ▁< ▁/ ▁doc > ▁xxbos
1,"' de ▁okudu . ▁xxup ▁h tv ▁xxmaj ▁wal es ' te ▁yayınlanan ▁"" jac ob ' s ▁xxmaj ▁ la d der "" ▁dizisinde ▁11 ▁yaşında ▁rol ▁aldı ▁ve ▁daha ▁sonra ▁xxmaj ▁royal ▁xxmaj ▁wel sh ▁xxmaj ▁college ▁of ▁xxmaj ▁music ▁& ▁xxmaj ▁drama ' da ▁okumaya ▁başladı ▁ve ▁2008 ▁yılında ▁oradan ▁mezun ▁oldu . ▁xxmaj ▁bar nard , ▁xxmaj ▁şubat ▁2009' da ▁xxmaj ▁tony ▁xxmaj ▁ödülleri ▁ödüllü ▁müzikal"
2,"▁önce ▁xxmaj ▁timur , ▁deniz ▁kalesi nin ▁yıkılması nı ▁ve ▁daha ▁önce ▁xxmaj ▁türklerin ▁elinde ▁durmuş ▁olan ▁ve ▁ akropol isi ▁koruyan ▁kara ▁kalesi nin ▁güçlendirilmesi ni ▁emretti . ▁xxmaj ▁i ̇ zmir ' in ▁fethi nin ▁akı bet inde ▁xxmaj ▁eski fo ça ' daki ▁xxmaj ▁ceneviz ▁karakolu ▁xxmaj ▁timur ' un ▁torunu ▁xxmaj ▁muhammed ▁xxmaj ▁sultan ▁xxmaj ▁mirza ' nın ▁ordularının ▁tehdidi ▁altındaydı . ▁xxmaj ▁yeni fo ça"
3,"▁xxmaj ▁bu ▁durumda , ▁her ▁topun ▁tek ▁tek ▁vur u lup ▁imha ▁edilmesi ▁gerekmektedir . ▁xxmaj ▁saldırı ▁gecesi ▁xxmaj ▁osmanlı ▁komutanlığı , ▁xxmaj ▁gelibolu ▁xxmaj ▁yarımadası ' nın ▁batı ▁sahillerinde , ▁olası ▁bir ▁çıkarma ▁hareketine ▁karşı ▁bazı ▁önlemler ▁almıştır . ▁xxmaj ▁bu ▁arada ▁saldırıya ▁uğrayan ▁tabya larda ▁tüm ▁gece ▁boyunca ▁hasar ların ▁giderilmesi ▁için ▁gereken ▁çalışmalar ▁yapıldı , ▁toplar ▁yeniden ▁ateşe ▁hazır ▁hale ▁getirildi . ▁xxmaj ▁boz an ▁hava ▁sebebiyle"
4,"▁geçirdiler ▁ve ▁hükümdar ▁xxmaj ▁hicaz ' a ▁kaçtı . ▁xxmaj ▁ku fe ▁xxmaj ▁mescid inde ▁kürsü ye ▁çıkan ▁xxmaj ▁muhtar , ▁xxmaj ▁ali ' nin ▁adalet ini ▁geri ▁getirmek ▁için ▁mücadelesini ▁sürdü r eceğini ▁açıkladı . ▁xxmaj ▁ay n ' ül ▁xxmaj ▁var da ▁xxmaj ▁muharebesi ▁sonrası ▁ilerleyen ▁xxmaj ▁emevî ▁xxmaj ▁ordusu ▁xxmaj ▁musul ' u ▁aldı ▁ve ▁xxmaj ▁ku fe ' ye ▁ilerledi . ▁xxmaj ▁muhtar , ▁xxmaj ▁yezid"


In [36]:
vocab = data.vocab

In [43]:
vocab.stoi["wikipedia"]

18063

In [39]:
wiki_words = set(vocab.itos)


In [85]:
wiki_words

{'▁istekleri',
 '▁almanya',
 '▁seçti',
 '▁ash',
 '▁kolesterol',
 'muyor',
 '▁yakalanan',
 '▁tarafının',
 '▁princeton',
 'millî',
 '▁geride',
 '▁kiril',
 'mog',
 'ru',
 '▁oynanır',
 '▁radyo',
 '▁lazio',
 'y',
 '▁kapandı',
 '▁yeryüzünde',
 '支',
 '▁yazılmış',
 '▁nehri',
 '▁musikisi',
 '▁davanın',
 '▁nazırlığı',
 '▁boynuzlu',
 'dari',
 '▁ışın',
 '▁kabilesi',
 '▁preston',
 '-6',
 '▁ettirildi',
 'ھ',
 '▁hizmetine',
 '▁molla',
 '▁yarımada',
 '▁açmıştır',
 '▁alışık',
 '苾',
 '▁butler',
 'ಯ',
 '▁uzan',
 '▁fosil',
 '▁pci',
 '▁microsoft',
 '字',
 '▁kullanabilme',
 '▁skeç',
 '▁bolu',
 '▁katerina',
 '(',
 '▁belarus',
 'lov',
 '▁benzet',
 '▁palermo',
 '▁adapt',
 '▁müziği',
 '▁ülker',
 'wikipedia',
 'ग',
 '▁yarıçap',
 '▁yüzyılın',
 '▁maddenin',
 '▁bristol',
 '▁mesajı',
 '▁yakıp',
 '▁aktarılması',
 'yy',
 '▁yıktı',
 '▁okunması',
 '2,8',
 '85',
 '▁kuşu',
 '▁amaçlayan',
 '▁düzeneği',
 'ayık',
 '▁çapındaki',
 '▁izah',
 '争',
 'product',
 '▁bozunma',
 'resmi',
 'landırır',
 'chor',
 'station',
 '▁kong',
 '▁k

In [44]:
"▁nesneleri" in wiki_words

True

In [45]:
"türkiye" in wiki_words

True

In [46]:
"futbol" in wiki_words

True

In [47]:
vocab.stoi["türkiye"]

3850

## Generating Fake Wiki

In [50]:
TEXT = "Türkiye bir"
# TEXT = "Temmuz ayı içerisinde"
N_WORDS = 40
N_SENTENCES = 2

In [51]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Türkiye bir ▁zamanlar ▁xxmaj ▁türkiye ' yi ▁oluşturan ▁xxmaj ▁kuzey ▁xxmaj ▁kıbrıs ▁xxmaj ▁türk ▁xxmaj ▁cumhuriyeti ' nden ▁sonra ▁en ▁büyük ▁ikinci ▁ülke dir . ▁xxmaj ▁türkiye ' nin ▁hiçbir ▁bölümünde ▁xxmaj ▁türkiye ' ye ▁bağlı ▁olmayan ▁xxmaj ▁türkiye ' nin ▁de
Türkiye bir ▁ilk e ▁imza ▁atması ▁xxmaj ▁türkiye ' de ▁ilk ▁kez ▁bir iktir ilmektedir . ▁xxmaj ▁bu ▁olayı ▁" üç lü ▁xxmaj ▁türkiye ▁ - ▁" kırı m ▁xxmaj ▁ yor tus u " ▁ - ▁xxmaj ▁i ̇ stanbul ▁xxmaj


In [52]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))

Türkiye bir ▁xxmaj ▁türk ▁vatandaşı dır . ▁xxmaj ▁türkiye ' de ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş ▁xxmaj ▁savaşı ' nın ▁ilk ▁yıllarında ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş ▁xxmaj ▁savaşı ' nın ▁en ▁önemli ▁askeri ▁ve ▁askeri ▁gücü ▁olan ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş ▁xxmaj ▁savaşı
Türkiye bir ▁xxmaj ▁türk ▁xxmaj ▁cumhuriyeti ' dir . ▁xxmaj ▁türkiye ' nin ▁xxmaj ▁türkiye ' ye ▁ilk ▁kez ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş ▁xxmaj ▁savaşı ' nda ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş ▁xxmaj ▁savaşı ' nda ▁yenilmesi nden ▁sonra ▁xxmaj ▁türk ▁xxmaj ▁kurtuluş


In [66]:
TEXT = "Türkiye'de kadın olmak"
# TEXT = "Temmuz ayı içerisinde"
N_WORDS = 40
N_SENTENCES = 2

In [67]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Türkiye'de kadın olmak ▁üzere ▁toplam ▁2 79 ▁kadın ▁ve ▁4 59 ▁kadın ▁olmak ▁üzere ▁toplam ▁12 3 ▁kadın ▁bu ▁görev le ▁ülkeyi ▁temsil ▁etmeye ▁başladı . ▁xxmaj ▁ ma tur i yan ▁xxmaj ▁okuma ▁xxmaj ▁ocağı ' nda ▁( şimdiki ▁xxmaj ▁türkiye ▁xxmaj
Türkiye'de kadın olmak ▁üzere ▁4 ▁ayrı ▁şahıs tan ▁oluşan ▁bir ▁ekip ▁bulunmaktadır . ▁xxmaj ▁türkiye ▁xxmaj ▁i ̇ şçi ▁xxmaj ▁partisi ▁ile ▁xxmaj ▁türkiye ▁xxmaj ▁büyük ▁xxmaj ▁millet ▁xxmaj ▁meclisi , ▁xxmaj ▁bakanlar ▁xxmaj ▁kurulu , ▁xxmaj ▁i ̇ ş ▁xxmaj ▁i ̇


In [72]:
TEXT = "Türkiye Almanya ilişkileri her zaman"
# TEXT = "Temmuz ayı içerisinde"
N_WORDS = 40
N_SENTENCES = 4

In [73]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Türkiye Almanya ilişkileri her zaman ▁önemli ▁bir ▁rol ▁oynamıştır . ▁xxmaj ▁türkiye , ▁xxmaj ▁türkiye ' nin ▁altı ▁ülkesinde n ▁biridir . ▁xxmaj ▁türkiye , ▁xxmaj ▁fransa ' nın ▁bir ▁parçası ▁olup ▁xxmaj ▁fransa ' nın ▁karşılık sız ▁olarak ▁başkenti ▁xxmaj ▁brüksel ' dir .
Türkiye Almanya ilişkileri her zaman ▁sağlam ▁ve ▁yavaş ken , ▁xxmaj ▁türkiye ▁xxmaj ▁cumhuriyeti , ▁xxmaj ▁türkiye ' nin ▁bu ▁sınırları ▁içerisinde ▁bulunan ▁üç ▁büyük ▁xxmaj ▁türkiye ' yi kapa dığı ▁bir ▁sırada ▁xxmaj ▁almanya ' nın ▁bir ▁parçası ▁olmuştur . ▁xxmaj ▁türkiye ' nin
Türkiye Almanya ilişkileri her zaman ▁daha ▁belirgin dir . ▁xxmaj ▁türkiye ' de ▁xxmaj ▁ankara ' nın ▁xxmaj ▁türkiye ' ye ▁ait ▁olmasından ▁yararlanılarak ▁kurulan ▁xxmaj ▁ankara ▁xxmaj ▁veteriner ▁xxmaj ▁bölümü , ▁xxmaj ▁türkiye ' deki ▁en ▁önemli ▁spor ▁dalları ▁arasındadır . ▁xxmaj ▁türkiye '
Türkiye Almanya ilişkileri her zaman ▁tek ▁bir ▁ülke ▁tarafından ▁yönetil mekteydi . ▁< ▁/ ▁doc > ▁xxbos ▁xxmaj ▁ er ya man ▁xxmaj ▁ er ya man , 

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [53]:
path

PosixPath('/home/ec2-user/.fastai/data/trwiki')

In [55]:
path_clas = path/'trmoviesentiment'
path_clas.ls()

[PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/tr_polarity.pos'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/Turkish_Movie_Sentiment.zip'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/tr_polarity.neg')]

In [56]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [57]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [58]:
df = pd.concat([pos_df,neg_df], sort=False)

In [59]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

# data_lm.save(f'{lang}_clas_databunch')

/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py:475: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ds.items = np.array(sum(e.map(self._encode_batch, partition_by_cores(ds.items, self.n_cpus)), []))


In [60]:
data_lm.save(f'{lang}_clas_databunch')

In [61]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [62]:
data_lm.show_batch()

idx,text
0,"sa ▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sav as ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gim ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al mis ti ▁onu ▁ ye tis tir mis ti"
1,le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁götür üyor ▁bask a ▁diya r lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁ in an ir mi siniz ▁bu ▁filmi ▁ ne ▁kadar ▁izledi gi
2,▁wallace  in ▁i sk ence ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁ unu ta mi yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁ in an di ktan ▁sonra ▁ ne leri ▁yap a bile ce gi ni ▁fark ▁etti m . ▁gerçekten ▁süper ▁film di . ▁halen ▁içi
3,▁uzun lug una ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gim ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nitel ig inde . o scar ▁al di gina ▁hiç ▁
4,▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁ anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izlemesi ▁gereken ▁bir ▁film . ben ▁10 ▁üzerinden ▁7 ▁verdi m ▁bu ▁benim ▁için ▁oldukça ▁çok ▁puan . ▁xxbos


# s3 backup HBA
s3://<folder>-ml/trlanguagemodel/models/

`
(base) $ aws s3 ls s3://<folder>-ml/trlanguagemodel/models/
2020-10-31 21:16:33  387655145 learner_mod_tr_spm_save.pkl.pth
2020-10-31 21:16:33  131193699 learner_modtr_spm.pkl
2020-10-31 21:16:33  129098532 learner_tr_spm_enc.pth
2020-10-31 21:16:33     577263 learner_vocab_tr_spm.pkl
2020-10-31 21:16:33  129219117 tr_wt.pth
2020-10-31 21:16:33     577263 tr_wt_vocab.pkl
`

`
s3://[golder]-ml/trlanguagemodel/docs/tr_databunch
`

`
(base)  $ aws s3 ls s3://[folder]-ml/trmoviesentiment/models/
2020-10-31 21:34:11  387655273 trfine_tuned.pth
2020-10-31 21:34:11  129098532 trfine_tuned_enc.pth
2020-10-31 21:46:14  170109634 trclas.pth
`

In [63]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [64]:
lr = 1e-3
lr *= bs/48

In [65]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.784773,4.122668,0.308103,00:40


In [74]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.130692,3.965839,0.329911,00:48
1,3.984847,3.865781,0.343036,00:48
2,3.760760,3.796683,0.354554,00:49
3,3.524284,3.779068,0.361429,00:49
4,3.303044,3.793252,0.364420,00:49


In [75]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [76]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py:475: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ds.items = np.array(sum(e.map(self._encode_batch, partition_by_cores(ds.items, self.n_cpus)), []))


/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [77]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [78]:
lr=2e-2
lr *= bs/48

In [89]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.519791,6.122978,0.484991,00:41
1,0.445498,0.367657,0.842402,00:42


In [ ]:
# learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [90]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.372276,0.521453,0.791745,00:43
1,0.348258,0.305345,0.869606,00:37


In [92]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.324016,0.318310,0.865854,00:54
1,0.298932,0.271153,0.891182,00:53
2,0.269719,0.256308,0.898687,00:56
3,0.252074,0.255850,0.900563,00:45


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [83]:
learn_c.save(f'{lang}clas')

## fin